***Player4 avec embedding des types avancé (graphes) (même si prend pas en compte le côté offensif), stats, nb pokémons vivants/morts***

In [1]:
#1 on se place 
import os
import ipynbname

chemin_notebook = ipynbname.path()
dossier_notebook = os.path.dirname(chemin_notebook)
os.chdir(dossier_notebook)
os.chdir('../..')
print("Répertoire actuel :", os.getcwd())

Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git


In [2]:
from Utils.embedding.type.type_embedding import *
from Players.Player4 import *

In [3]:
############## DEFINIR L'EMBEDDING ##################
import numpy as np
from gymnasium.spaces import Space, Box
from poke_env.player import Gen4EnvSinglePlayer
from poke_env.data import GenData
from poke_env.environment.abstract_battle import AbstractBattle
import torch
from gymnasium.spaces import Discrete, Box

# Initialiser GenData pour la génération souhaitée (par exemple, génération 4)
gen_data = GenData.from_gen(4)

# Accéder au tableau des types
type_chart = gen_data.type_chart


class Player4Training(Gen4EnvSinglePlayer): #Classe pour entrainer le modèle 4. 
    def __init__(self,model_path=None, **kwargs):
        super().__init__(**kwargs)
        if model_path is not None :
            #self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
            print(f"📥 Modèle chargé depuis {model_path}")

        self.action_space = Discrete(9)  # ✅ attribut classique
        self.my_team = None
        self.my_team_index = {}
        self.opponent_team = None
        self.opponent_team_index = {}
    
    #Toujours mêmes valeurs de reward
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle)  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        if battle.turn == 1:
            self.my_team, self.my_team_index = init_my_team(battle)
            self.opponent_team, self.opponent_team_index = init_opponent_team(battle)
        else : 
            self.my_team = update_my_team(battle, self.my_team, self.my_team_index)
            self.opponent_team, self.opponent_team_index = update_opponent_team(battle, self.opponent_team, self.opponent_team_index)

        

        # Final vector with 10 components
        final_vector = np.concatenate(
            [
                moves_real_power,
                self.my_team,
                self.opponent_team
            ]
        )
        final_tensor = torch.tensor(final_vector,dtype=torch.float32)
        #print("taille tensor state : ",final_tensor.shape)
        #print("index mon équipe : ",self.my_team_index)
        #print("index equipe adverse : ",self.opponent_team_index)
        return final_tensor

    def describe_embedding(self) -> Space:
        low = (
            [-1] * 4 +          # real power
            [0] * 72 +         # my team types
            [0] * 72           # opponent team types
        )
        high = (
            [3] * 4 +           # real power
            [1] * 72 +         # my team types
            [1] * 72           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
    
    def action_to_move(self, action: int, battle: AbstractBattle):
        order = super().action_to_move(action, battle)
        return order

In [4]:
########### DEFINIR MODELE ##################
import torch
import torch.nn as nn
import torch.nn.functional as F

#Réseau perceptron à deux couche cachée, sortie linéaire, f activation = Relu,
class DQNModel(nn.Module):
    def __init__(self, input_dim, n_actions):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.out = nn.Linear(128, n_actions)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.out(x)


In [5]:
class DQNModelAdversaire(nn.Module):
    def __init__(self, input_dim, n_actions):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=3, padding=1)
        self.conv2= nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.out = nn.Linear(32, n_actions)

    def forward(self, x):
        real_power = x[:4]
        types = x[4:] 
        real_power_padded = F.pad(real_power, (0, 14))  # [17]
        real_power_padded = real_power_padded.unsqueeze(0)  # [1, 17] 
        sequence = torch.cat([real_power_padded, types], dim=0)
        sequence = sequence.T.unsqueeze(0)     
        types = types.view(12, 17) 
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return self.out(x)

In [6]:

#opponent = AdversarialTrainPlayer(battle_format="gen4randombattle")
from stable_baselines3.common.vec_env import DummyVecEnv

# Réinstancier l'env
train_env_raw = Player4Training(
    battle_format="gen4randombattle",
    opponent=RandomPlayer(battle_format="gen4randombattle"),
    start_challenging=True
)

train_env = DummyVecEnv([lambda: train_env_raw])  # Redéfini bien après reset

In [7]:
env = train_env

model = DQN(
    policy="MlpPolicy",
    env=env,
    learning_rate=2.5e-4,
    buffer_size=10000,
    learning_starts=1000,
    batch_size=32,
    gamma=0.5,
    train_freq=1,
    target_update_interval=1,
    exploration_fraction=1.0,
    exploration_final_eps=0.05,
    policy_kwargs=dict(activation_fn=nn.ReLU, net_arch=[128, 64])
)


2025-04-14 18:12:44,324 - RandomPlayer 1 - CRITICAL - Error message received: |nametaken|RandomPlayer 1|Someone is already using the name "RandomPlayer 1".
2025-04-14 18:12:44,324 - RandomPlayer 1 - ERROR - Unhandled exception raised while handling message:
|nametaken|RandomPlayer 1|Someone is already using the name "RandomPlayer 1".
Traceback (most recent call last):
  File "/Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git/showdown_4/lib/python3.11/site-packages/poke_env/ps_client/ps_client.py", line 164, in _handle_message
    raise ShowdownException("Error message received: %s", message)
poke_env.exceptions.ShowdownException: ('Error message received: %s', '|nametaken|RandomPlayer 1|Someone is already using the name "RandomPlayer 1".')


In [8]:
#steps = 1000000
#callback = CustomTQDMCallback(total_timesteps = steps, check_freq=1000, verbose=0)
#model.learn(
#    total_timesteps=steps,
#    callback=callback
#)
#model.save("Players/player_4_dynamic")

In [9]:
class Player4EvalvsBot(Gen4EnvSinglePlayer):

    def __init__(self, model_path="Players/player_4", **kwargs):
        super().__init__(**kwargs)
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print(f"📥 Modèle chargé depuis {model_path}")
        print("Répertoire actuel :", os.getcwd())
        self.my_team = None
        self.my_team_index = {}
        self.opponent_team = None
        self.opponent_team_index = {}
        
        #Toujours mêmes valeurs de reward
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle)  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        if battle.turn == 1:
            self.my_team, self.my_team_index = init_my_team(battle)
            self.opponent_team, self.opponent_team_index = init_opponent_team(battle)
        else : 
            self.my_team = update_my_team(battle, self.my_team, self.my_team_index)
            self.opponent_team, self.opponent_team_index = update_opponent_team(battle, self.opponent_team, self.opponent_team_index)

        

        # Final vector with 10 components
        final_vector = np.concatenate(
            [
                moves_real_power,
                self.my_team,
                self.opponent_team
            ]
        )
        final_tensor = torch.tensor(final_vector,dtype=torch.float32)
        return final_tensor

    def describe_embedding(self) -> Space:
        low = (
            [-1] * 4 +          # real power
            [0] * 72 +         # my team types
            [0] * 72           # opponent team types
        )
        high = (
            [3] * 4 +           # real power
            [1] * 72 +         # my team types
            [1] * 72           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
    
    def action_to_move(self, action, battle):
        print('my_team_index : ',self.my_team_index)
        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)

        #print(f"🔢 DQN → action={action} | #moves={len(moves)} | #switches={len(switches)} | total={total_actions}")

        if 0 <= action < len(moves):
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            return self.create_order(switches[action - len(moves)])
        else:
            #print("❌ Action hors bornes, choix d'un move aléatoire")
            return self.choose_random_move(battle)
        
    def predict(self, obs):
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        q_values = self.model.q_net(obs_tensor)
        print(f"📊 Q-values : {q_values.detach().numpy().flatten()}")
        action = int(torch.argmax(q_values).item())
        return action

***EVAL***

In [10]:
############DEFINIR L ADVERSAIRE#####################
from Utils.embedding.old.old_embedding import *
#Instancier l'encodeur du type 
from Utils.embedding.old.type_autoencodeur import TypeAutoencoder

encodeur_type = TypeAutoencoder(encoded_size=4)

# Charger les poids sauvegardés
encodeur_type.load_state_dict(torch.load("Utils/embedding/old/type_autoencoder.pth", map_location='mps'))
encodeur_type.eval()

TypeAutoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=17, out_features=12, bias=True)
    (1): ReLU()
    (2): Linear(in_features=12, out_features=4, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=4, out_features=12, bias=True)
    (1): ReLU()
    (2): Linear(in_features=12, out_features=17, bias=True)
    (3): Sigmoid()
  )
)

In [11]:
class AdversarialTrainPlayer(RandomPlayer): # Adversaire de Player4 lors de l'entrainement

    def __init__(self, model_path="Players/embedding_2", **kwargs):
        super().__init__(**kwargs)
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print(f"📥 Modèle chargé depuis {model_path}")

    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        pokemon_types_compressed = []
        #Pokemon types  
        pokemon_types = obtain_pokemon_types(battle)
        pokemon_types = torch.tensor(obtain_pokemon_types(battle)).view(12, 17)  # 6 pokés par team = 12 vecteurs
        with torch.no_grad():
            for i in range(12) :
                vec = pokemon_types[i].unsqueeze(0)
                encoded = encodeur_type.encoder(vec.float()) 
                pokemon_types_compressed.append(encoded.squeeze(0))

        # Final vector with 10 components
        compressed_flat = torch.cat(pokemon_types_compressed).numpy()
        final_vector = np.concatenate(
            [
                moves_real_power,
                compressed_flat,
            ]
        )

        return np.float32(final_vector)
    
    #action_to_move suppr
        
    def predict(self, obs):
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        q_values = self.model.q_net(obs_tensor)
        print(f"📊 Q-values : {q_values.detach().numpy().flatten()}")
        action = int(torch.argmax(q_values).item())
        return action
    
    def choose_move(self, battle):
        obs = self.embed_battle(battle)
        device = "mps" if torch.backends.mps.is_available() else "cpu"
        obs_tensor = torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
        with torch.no_grad():
            q_values = self.model.q_net(obs_tensor)
            action = int(torch.argmax(q_values).item())

        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)

        if 0 <= action < len(moves):
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            return self.create_order(switches[action - len(moves)])
        else:
            return self.choose_random_move(battle)

In [12]:
opponent = AdversarialTrainPlayer(battle_format="gen4randombattle")
eval_env = Player4EvalvsBot(
    battle_format="gen4randombattle", opponent=opponent, start_challenging=True
)

📥 Modèle chargé depuis Players/embedding_2
📥 Modèle chargé depuis Players/player_4
Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git


In [13]:
n_eval_episodes = 1
rewards = []
wins = 0

obs, _ = eval_env.reset()
for _ in tqdm(range(n_eval_episodes)):
    done = False
    total_reward = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, _ = eval_env.step(action)
        done = terminated or truncated

        total_reward += reward
        if done and reward > 0:
            wins += 1

    rewards.append(total_reward)
    obs, _ = eval_env.reset()

print(f"✅ {wins} victoires sur {n_eval_episodes} matchs")
print(f"🎯 Reward moyen : {sum(rewards) / len(rewards):.2f}")

  0%|          | 0/1 [00:00<?, ?it/s]

my_team_index :  {'metagross': 0, 'primeape': 1, 'snorlax': 2, 'vileplume': 3, 'seaking': 4, 'mothim': 5}
my_team_index :  {'metagross': 0, 'primeape': 1, 'snorlax': 2, 'vileplume': 3, 'seaking': 4, 'mothim': 5}
my_team_index :  {'metagross': 0, 'primeape': 1, 'snorlax': 2, 'vileplume': 3, 'seaking': 4, 'mothim': 5}
my_team_index :  {'metagross': 0, 'primeape': 1, 'snorlax': 2, 'vileplume': 3, 'seaking': 4, 'mothim': 5}
my_team_index :  {'metagross': 0, 'primeape': 1, 'snorlax': 2, 'vileplume': 3, 'seaking': 4, 'mothim': 5}
my_team_index :  {'metagross': 0, 'primeape': 1, 'snorlax': 2, 'vileplume': 3, 'seaking': 4, 'mothim': 5}
my_team_index :  {'metagross': 0, 'primeape': 1, 'snorlax': 2, 'vileplume': 3, 'seaking': 4, 'mothim': 5}
my_team_index :  {'metagross': 0, 'primeape': 1, 'snorlax': 2, 'vileplume': 3, 'seaking': 4, 'mothim': 5}
my_team_index :  {'metagross': 0, 'primeape': 1, 'snorlax': 2, 'vileplume': 3, 'seaking': 4, 'mothim': 5}
my_team_index :  {'metagross': 0, 'primeape': 

✅ 0 victoires sur 1 matchs
🎯 Reward moyen : -35.56


Pour que je puisse l'affronter : 

In [14]:
from Utils.DQ_simple import DQ_simple 
class Player4vsHuman(DQ_simple) :
    def __init__(self, model_path = "Players/player_4", battle_format="gen4randombattle"):
        super().__init__(battle_format=battle_format)
        
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print("📥 Modèle DQN chargé :", self.model)
        self.my_team = None
        self.my_team_index = {}
        self.opponent_team = None
        self.opponent_team_index = {}
    
    def embed_battle(self, battle)  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        if battle.turn == 1:
            self.my_team, self.my_team_index = init_my_team(battle)
            self.opponent_team, self.opponent_team_index = init_opponent_team(battle)
        else : 
            self.my_team = update_my_team(battle, self.my_team, self.my_team_index)
            self.opponent_team, self.opponent_team_index = update_opponent_team(battle, self.opponent_team, self.opponent_team_index)

        

        # Final vector with 10 components
        final_vector = np.concatenate(
            [
                moves_real_power,
                self.my_team,
                self.opponent_team
            ]
        )
        final_tensor = torch.tensor(final_vector,dtype=torch.float32)
        return final_tensor

    def choose_move(self, battle):
        print("👉 nouveau tour !")
        print("état de ma team : ",self.my_team)
        print("état de la team adverse : ",self.opponent_team)

        # Obtenir l'observation de l'état du combat
        obs = self.embed_battle(battle)
        #print("📊 Observation de l'état :", obs)

        # Prédire l'action avec le modèle DQN
        action = int(self.model.predict(obs, deterministic=True)[0])
        
        #print("🎯 Action choisie par DQN :", action)
        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)
        self.create_order.dynamax = False

        if 0 <= action < len(moves):
            print(self.create_order(moves[action]))
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            print(self.create_order(switches[action - len(moves)]))
            return self.create_order(switches[action - len(moves)])
        else:
            print("XXXX   : Random Moove ")
            a = self.choose_random_move(battle)
            print(a)
            return a

In [15]:
bot_player = Player4vsHuman()

#await bot_player.send_challenges("chewba", n_challenges=1)

📥 Modèle DQN chargé : <stable_baselines3.dqn.dqn.DQN object at 0x175c18b50>
📥 Modèle DQN chargé : <stable_baselines3.dqn.dqn.DQN object at 0x3157059d0>
